In [1]:
# !pip install duckdb==1.4.3
import duckdb
import pandas as pd

# DB_PATH = r'''c:\\users\\55119\\desktop\\dev\\airflow\\airflow_demo\\data\\meu_data_warehouse.db'''
con = duckdb.connect(r'C:\Users\55119\Desktop\DEV\AIRFLOW\AIRFLOW_DEMO\data\meu_data_warehouse.db')

C:\Users\55119\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
# CRM
df_crm =con.sql("""
SELECT user_id as ID_USUARIO,
lower(email) as EMAIL,
CAST(REGEXP_REPLACE(phone, '[^0-9]', '', 'g') AS VARCHAR) AS NUMERO_TELEFONE,
CAST(conversion_at AS TIMESTAMP) AT TIME ZONE 'America/Sao_Paulo' AS DATA_CONVERSAO
from crm_user
""").df()
df_crm.head(2)

,ID_USUARIO,EMAIL,NUMERO_TELEFONE,DATA_CONVERSAO
0,u_1000,camila.nunes@gmail.com,5540934523917,2023-10-15 16:00:00-03:00
1,u_1001,julia.silva@outlook.com,5560989882249,NaT


In [71]:
#EMAIL
df_email = con.sql("""
SELECT event_id AS ID_EVENTO,
lower(user_email) as EMAIL_USUARIO,
CAST(event_timestamp AS TIMESTAMP) AT TIME ZONE 'America/Sao_Paulo' AS DATA_EVENTO,
event_type as TIPO_EVENTO,
REPLACE(json_extract(message_details, '$.campaign_code'), '"','') AS CODIGO_CAMPANHA
--json_extract(message_details, '$.journey_id') AS journey_id,
--json_extract(message_details, '$.template') AS template,
from email_logs
WHERE
json_valid(message_details) = TRUE
""").df()

In [72]:
df_email.head(2)

,ID_EVENTO,EMAIL_USUARIO,DATA_EVENTO,TIPO_EVENTO,CODIGO_CAMPANHA
0,evt_0b8fe40d,camila.nunes@gmail.com,2023-09-05 19:40:02-03:00,open,CASHBACK_OFFER
1,evt_c2349fe9,camila.nunes@gmail.com,2023-09-13 19:40:02-03:00,open,CREDIT_LIMIT_UP


In [24]:
# WPP
df_wpp =con.sql("""
SELECT message_id as ID_MENSAGEM,
CAST(phone_clean as VARCHAR) as NUMERO_TELEFONE,
CAST(sent_at_brt AS TIMESTAMP) AT TIME ZONE 'America/Sao_Paulo' AS DATA_ENVIO,
status as STATUS_ENVIO,
TRIM(REPLACE(REGEXP_EXTRACT(campaign_tag, '-\s*([^-\s]+)\s*-'), '-', '')) as TAG_CAMPANHA
from whatsapp_provider
""").df()
df_wpp.head(2)

,ID_MENSAGEM,NUMERO_TELEFONE,DATA_ENVIO,STATUS_ENVIO,TAG_CAMPANHA
0,wa_17bcbdb1,5523937224932,2023-04-28 06:37:25-03:00,failed,BLK_FRIDAY_23
1,wa_9bf44f0a,5523937224932,2023-10-18 11:52:52-03:00,read,CREDIT_LIMIT_UP


In [ ]:
df_gold =con.sql("""
WITH WPP_DATA AS (
SELECT message_id as ID_MENSAGEM,
phone_clean as NUMERO_TELEFONE,
CAST(sent_at_brt AS TIMESTAMP) AT TIME ZONE 'America/Sao_Paulo' AS DATA_ENVIO,
status as STATUS_ENVIO,
campaign_tag as TAG_CAMPANHA
from whatsapp_provider
),
EMAIL_DATA AS (
SELECT event_id AS ID_EVENTO,
lower(user_email) as EMAIL_USUARIO,
CAST(event_timestamp AS TIMESTAMP) AT TIME ZONE 'America/Sao_Paulo' AS DATA_EVENTO,
event_type as TIPO_EVENTO,
REPLACE(json_extract(message_details, '$.campaign_code'), '"','') AS CODIGO_CAMPANHA
--json_extract(message_details, '$.journey_id') AS journey_id,
--json_extract(message_details, '$.template') AS template,
from email_logs
WHERE
json_valid(message_details) = TRUE
),
USER_DATA AS (
SELECT user_id as ID_USUARIO,
lower(email) as EMAIL,
REGEXP_REPLACE(phone, '[^0-9]', '', 'g') AS NUMERO_TELEFONE,
CAST(conversion_at AS TIMESTAMP) AT TIME ZONE 'America/Sao_Paulo' AS DATA_CONVERSAO
from crm_user
)

SELECT ID_USUARIO, EMAIL, NUMERO_TELEFONE , DATA_CONVERSAO,
-- Definindo pesos (○ Peso 1 (Maior): WhatsApp Read, Peso 2 (Médio): Email Click, 3 (Menor): Email Open)
CASE WHEN WPP_DATA.STATUS_ENVIO = 'read' THEN 1
WHEN EMAIL_DATA.TIPO_EVENTO = 'click' THEN 2
WHEN EMAIL_DATA.TIPO_EVENTO = 'open' THEN 3
ELSE 5
END AS PESO_INTERACAO
FROM USER_DATA
LEFT JOIN EMAIL_DATA ON USER_DATA.EMAIL = EMAIL_DATA.EMAIL_USUARIO
-- Interações ocorridas nos 7 dias anteriores à conversão
AND EMAIL_DATA.DATA_EVENTO BETWEEN USER_DATA.DATA_CONVERSAO - INTERVAL '7 DAYS' AND USER_DATA.DATA_CONVERSAO

LEFT JOIN WPP_DATA ON USER_DATA.NUMERO_TELEFONE = WPP_DATA.NUMERO_TELEFONE
-- Interações ocorridas nos 7 dias anteriores à conversão
AND WPP_DATA.DATA_ENVIO BETWEEN USER_DATA.DATA_CONVERSAO - INTERVAL '7 DAYS' AND USER_DATA.DATA_CONVERSAO

WHERE DATA_CONVERSAO IS NOT NULL
QUALIFY ROW_NUMBER() OVER (PARTITION BY USER_DATA.ID_USUARIO ORDER BY PESO_INTERACAO ASC, COALESCE(DATA_ENVIO, DATA_EVENTO) DESC) = 1

""").df()
df_gold.head(2)

,ID_USUARIO,EMAIL,NUMERO_TELEFONE,DATA_CONVERSAO,ID_EVENTO,EMAIL_USUARIO,DATA_EVENTO,TIPO_EVENTO,CODIGO_CAMPANHA,ID_MENSAGEM,NUMERO_TELEFONE_1,DATA_ENVIO,STATUS_ENVIO,TAG_CAMPANHA,PESO_INTERACAO
0,u_1050,maria.silva@gmail.com,5541926402267,2023-06-07 20:27:11-03:00,evt_6d3b86e4,maria.silva@gmail.com,2023-06-04 20:27:11-03:00,sent,CASHBACK_OFFER,None,<NA>,NaT,None,None,5
1,u_1074,julia.santos@gmail.com,5579928641745,2023-08-22 13:15:42-03:00,evt_7ac46682,julia.santos@gmail.com,2023-08-16 13:15:42-03:00,open,WELCOME_JOURNEY,None,<NA>,NaT,None,None,3


In [9]:
df_gold

,ID_USUARIO,EMAIL,NUMERO_TELEFONE,DATA_CONVERSAO,ID_EVENTO,EMAIL_USUARIO,DATA_EVENTO,TIPO_EVENTO,CODIGO_CAMPANHA,ID_MENSAGEM,NUMERO_TELEFONE_1,DATA_ENVIO,STATUS_ENVIO,TAG_CAMPANHA,PESO_INTERACAO
0,u_1050,maria.silva@gmail.com,5541926402267,2023-06-07 20:27:11-03:00,evt_6d3b86e4,maria.silva@gmail.com,2023-06-04 20:27:11-03:00,sent,CASHBACK_OFFER,None,<NA>,NaT,None,None,5
1,u_1074,julia.santos@gmail.com,5579928641745,2023-08-22 13:15:42-03:00,evt_7ac46682,julia.santos@gmail.com,2023-08-16 13:15:42-03:00,open,WELCOME_JOURNEY,None,<NA>,NaT,None,None,3
2,u_1178,pedro.pereira@gmail.com,5513956777995,2023-11-20 01:23:44-03:00,evt_8bde567d,pedro.pereira@gmail.com,2023-11-13 01:23:44-03:00,click,LOAN_OFFER,None,<NA>,NaT,None,None,2
3,u_1198,ana.nunes@gmail.com,5532945107804,2023-12-09 10:21:37-03:00,evt_288c9d82,ana.nunes@gmail.com,2023-12-02 10:21:37-03:00,sent,CREDIT_LIMIT_UP,None,<NA>,NaT,None,None,5
4,u_1251,marcos.alves@outlook.com,5566987924395,2023-05-20 14:56:54-03:00,evt_d1005abc,marcos.alves@outlook.com,2023-05-19 14:56:54-03:00,sent,RETENTION_PHASE1,None,<NA>,NaT,None,None,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,u_1228,maria.oliveira@hotmail.com,5540965822375,2023-07-19 03:19:32-03:00,None,None,NaT,None,None,None,<NA>,NaT,None,None,5
208,u_1273,marcos.costa@outlook.com,5570925403296,2023-08-02 18:49:07-03:00,evt_d9532539,marcos.costa@outlook.com,2023-07-30 18:49:07-03:00,open,WELCOME_JOURNEY,None,<NA>,NaT,None,None,3
209,u_1284,patricia.lima@outlook.com,5514987047319,2023-07-28 05:46:23-03:00,evt_78d966f8,patricia.lima@outlook.com,2023-07-26 05:46:23-03:00,open,RETENTION_PHASE1,None,<NA>,NaT,None,None,3
210,u_1316,camila.nunes@outlook.com,5576948409519,2023-08-11 19:34:26-03:00,None,None,NaT,None,None,None,<NA>,NaT,None,None,5


In [19]:

# # 3. Listar tabelas por schema
# con.sql("""
# SELECT table_schema, table_name, table_type
# FROM information_schema.tables
# ORDER BY table_schema, table_name
# """).df()


In [2]:
#DF GOLD
# # 3. Listar tabelas por schema
con.sql("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
ORDER BY table_schema, table_name
""").df()


,table_schema,table_name,table_type
0,main,crm_user,BASE TABLE
1,main,email_logs,BASE TABLE
2,main,gold_unified_view,BASE TABLE
3,main,silver_crm_user,BASE TABLE
4,main,silver_email_logs,BASE TABLE
5,main,silver_whatsapp_provider,BASE TABLE
6,main,whatsapp_provider,BASE TABLE


In [ ]:
FILE_CONFIG = {
    "file": "/data/raw_sfmc_email_logs.csv",
    # "file": "/opt/airflow/data/raw_sfmc_email_logs.csv",
    "table": "email_logs"

}

In [38]:
CSV_PATH = r'C:\Users\55119\Desktop\DEV\AIRFLOW\AIRFLOW_DEMO\data\raw_sfmc_email_logs.csv'
VALID_OUTPUT_PATH = r"C:\Users\55119\Desktop\DEV\AIRFLOW\AIRFLOW_DEMO\data\email_logs_valid.csv"

In [39]:
import csv
import json
import logging

def validate_json_field():
    valid_rows = []
    invalid_count = 0

    with open(CSV_PATH, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        
        for line_number, row in enumerate(reader, start=1):
            try:
                json.loads(row["message_details"])
                valid_rows.append(row)
            except json.JSONDecodeError as e:
                invalid_count += 1
                logging.error(f"Invalid JSON at line {line_number}: {row['message_details']} | Error: {e}")
    if valid_rows:
        with open(VALID_OUTPUT_PATH, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=valid_rows[0].keys())
            writer.writeheader()
            writer.writerows(valid_rows)

    logging.info(f"Valid rows: {len(valid_rows)}")
    logging.info(f"Invalid rows discarded: {invalid_count}")
    return len(valid_rows)

In [40]:
validate_json_field()

ERROR:root:Invalid JSON at line 5: {"campaign_code": "BROKEN_JSON", "oops":  | Error: Expecting value: line 1 column 42 (char 41)
ERROR:root:Invalid JSON at line 18: {"campaign_code": "BROKEN_JSON", "oops":  | Error: Expecting value: line 1 column 42 (char 41)
ERROR:root:Invalid JSON at line 31: {"campaign_code": "BROKEN_JSON", "oops":  | Error: Expecting value: line 1 column 42 (char 41)
ERROR:root:Invalid JSON at line 123: {"campaign_code": "BROKEN_JSON", "oops":  | Error: Expecting value: line 1 column 42 (char 41)
ERROR:root:Invalid JSON at line 147: {"campaign_code": "BROKEN_JSON", "oops":  | Error: Expecting value: line 1 column 42 (char 41)
ERROR:root:Invalid JSON at line 181: {"campaign_code": "BROKEN_JSON", "oops":  | Error: Expecting value: line 1 column 42 (char 41)
ERROR:root:Invalid JSON at line 196: {"campaign_code": "BROKEN_JSON", "oops":  | Error: Expecting value: line 1 column 42 (char 41)
ERROR:root:Invalid JSON at line 202: {"campaign_code": "BROKEN_JSON", "oops":  |

1486